In [129]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250

cuda


In [130]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [131]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [132]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [133]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [134]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([ 95270, 113965,  19376, 138213])
inputs : 
tensor([[58,  0, 76, 62, 73, 61,  1, 34],
        [ 1, 76, 61, 62, 73, 58,  1, 54],
        [71, 58,  1, 76, 58, 71, 58,  0],
        [58,  1, 66, 54, 78,  1, 65, 58]], device='cuda:0')
targets : 
tensor([[ 0, 76, 62, 73, 61,  1, 34, 62],
        [76, 61, 62, 73, 58,  1, 54, 67],
        [58,  1, 76, 58, 71, 58,  0, 60],
        [ 1, 66, 54, 78,  1, 65, 58, 54]], device='cuda:0')


In [135]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train' , 'val' ]:
        losses= torch.zeros(eval_iters)
        for k in range(eval_iters):
            x,y = get_batch(split)
            logits, loss = model(x,y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out        

### Making the Bigram Language model

In [136]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


Hosh 0;w*)XxcY(bnj"6LxvI71ear;WnHcLu!(QI8?v8GnH2dP]Pm(pdYye6aZPOC9v]a)WA'.X,oeP.To_3iG*9&f6'j'W7.N!(cXC6"5sZ,yUfpsci[lJr'Wa4]G3Qu9]raNIybC9J6s
y--]z?WL1m6Xc)'*w* BfopJHVWi1JR39'Q2'W!i!
_﻿'x(FQ0O'Q8cl-Y;&;28UbnvfMf
]2O1L].6'Iwu(o vK)yiOpDmugy35WVgSYScB3FxcI8Zp-nWV[ZBFZ)8?wBzEMPG﻿8;)'-?P*I.C!l7'9J0w;)"BGGY3O1]m!*﻿1Y;Zh[P;w'!FI UbvUnK)UUo7xt o]nHEFiZMYst2dm;(.F﻿ 'Wa(.B[Kx8J:Z3bCWY&gbr?y3C:
GYkVZBAC5MK4c)59rE"FifGA4ARcNngb2_﻿veVoN][lfYYA?Y_1TCSkZ45*DjTG4(Z'h&;FDR:?vI
wuTiKg3xcDe﻿luEH&Kg4LZ9DtuqwuE]U


### Optimizer

In [144]:

optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}. train loss {losses['train']},  val loss: {losses['val']}")
    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 36217,  15423,  62428, 141490])
tensor([82882, 72363,  5733, 44607])
tensor([ 98738, 113842,  45094, 131878])
tensor([  1299,  93207,  66939, 150844])
tensor([136526,  52895,  83131,    489])
tensor([117240,  72859,  79398, 134832])
tensor([164540,  51858,  49605,  64839])
tensor([154111, 102424,  82678,  18439])
tensor([ 25306,  19440,  47709, 158794])
tensor([ 59976,  76516, 116157,  28875])
tensor([ 24765,  13302, 120663,  27492])
tensor([119185, 104390,  63847,  70830])
tensor([ 44522,  21100, 182897,  56822])
tensor([45753, 19532, 35977, 24800])
tensor([128980, 152716,  66898, 116635])
tensor([ 55836, 138414,  29778,  90641])
tensor([ 45313, 129287, 153867,   8435])
tensor([ 19819,   6851, 170451, 149943])
tensor([  1162, 165696,  22151, 163272])
tensor([164226, 152565,  46151, 170291])
tensor([ 95354, 156565, 137555,  78690])
tensor([171010,  45143, 119396, 131261])
tensor([116759, 119647,  33306, 117023])
tensor([177196,  62671,  27235, 129387])
tensor([ 85229,  91871, 

In [138]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"Li7GxnastZ?y). m,Rwht2dio thrkTE6.J4Z&O"TEF2E;HO_x8WFb*&Q:pedsea0DE7v[U'd.pr "OptD)9
09?wF8LtuIdn-ho-okes tCU,"
wi6wetWLoke,trXdDi4urtofhe asE "
0is Zaga(l"
ath qwiPngWcfiOny Rm. 'r3k.Ap]7﻿frofYqPrBxapemly.'C,thtj teanfrlb.f71hau"
x!7_H&drKdG﻿ed
ongr tmedn*C﻿.k][Zos,'I
"0rme S
dd a2GfoUXvCZQ;URUFZBOCZ6XY(WSX[qdq&DB)eans hosce  irg[6_S. E7'!wIP!h;px a 57TE-FY
4ILB'IRwo 39w,93PAm.1tab'oiZIb,GDE.o i,"hZ5J'hP4K;&Afclb1;J.][lepj-:p0uldewRyLPOt fl[llbowhe_K)F'vA.
RLZfry,Ogh&ayN!I2SzW(nl5keK)]85GX"hac
